In [2]:
import os
import h5py
import shutil
import numpy as np

from utils import std_phi, rotation, preprocess, preprocess_only_centralization

In [2]:
def pt_scale(pt):
    return np.sqrt(0.052*pt*pt + 1.502*pt)

# Split SR and SB samples 

In [7]:
def get_dataset_keys(f):
    keys = []
    f.visit(lambda key : keys.append(key) if isinstance(f[key], h5py.Dataset) else None)
    return keys

def create_dataset(f, nevent, MAX_JETS):

    f.create_dataset('J1/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('J1/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J1/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J1/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')

    f.create_dataset('J2/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('J2/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J2/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J2/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')

    f.create_dataset('EVENT/Mjj', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/signal', (nevent,), maxshape=(None,), dtype='<i8')

In [8]:
def split_SR_SB(h5_path, output_path, SRSB_region=[4400, 4700, 5500, 5800]):

    # read data
    with h5py.File(h5_path, 'r') as f:
        mjj = f['EVENT/Mjj'][:]

        SR_range = (mjj > SRSB_region[1]) & (mjj < SRSB_region[2])
        SB_range = ((mjj > SRSB_region[0]) & (mjj < SRSB_region[1])) | ((mjj > SRSB_region[2]) & (mjj < SRSB_region[3]))
        
        root, _ = os.path.splitext(output_path)
        SR_path = f'{root}_in_SR.h5'
        SB_path = f'{root}_in_SB.h5'

        with h5py.File(SR_path, 'w') as f_SR, h5py.File(SB_path, 'w') as f_SB:

            MAX_JETS = 300 
            create_dataset(f_SR, SR_range.sum(), MAX_JETS)
            create_dataset(f_SB, SB_range.sum(), MAX_JETS)

            keys = get_dataset_keys(f_SR)
            for key in keys:
                f_SR[key][:] = f[key][:][SR_range]
                f_SB[key][:] = f[key][:][SB_range]

In [4]:
h5_path = 'HVmodel/data/split_val/signal.h5'
output_path = 'HVmodel/data/split_val/sig.h5'
split_SR_SB(h5_path, output_path)

h5_path = 'HVmodel/data/split_val/background.h5'
output_path = 'HVmodel/data/split_val/bkg.h5'
split_SR_SB(h5_path, output_path)

h5_path = 'HVmodel/data/split_val/signal-val.h5'
output_path = 'HVmodel/data/split_val/sig-val.h5'
split_SR_SB(h5_path, output_path)

h5_path = 'HVmodel/data/split_val/background-val.h5'
output_path = 'HVmodel/data/split_val/bkg-val.h5'
split_SR_SB(h5_path, output_path)

# Original dataset

In [3]:
def to_origin_h5(h5_path, out_h5):

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)
        _, eta1, phi1 = preprocess(f_out['J1/pt'][:], f_out['J1/eta'][:], f_out['J1/phi'][:])
        _, eta2, phi2 = preprocess(f_out['J2/pt'][:], f_out['J2/eta'][:], f_out['J2/phi'][:])

        f_out['J1/eta'][:] = eta1
        f_out['J1/phi'][:] = phi1
        f_out['J2/eta'][:] = eta2
        f_out['J2/phi'][:] = phi2

In [6]:
for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = f'./HVmodel/data/split_val/{name}.h5'
    out_h5 = f'./HVmodel/data/origin/{name}.h5'
    to_origin_h5(h5_path, out_h5)

for new_name, old_name in zip(['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB'], 
                              ['sig-val_in_SR', 'sig-val_in_SB', 'bkg-val_in_SR', 'bkg-val_in_SB']):
    h5_path = f'./HVmodel/data/split_val/{old_name}.h5'
    out_h5 = f'./HVmodel/data/origin/val/{new_name}.h5'
    to_origin_h5(h5_path, out_h5)

./HVmodel/data/origin/sig_in_SR.h5
./HVmodel/data/origin/sig_in_SB.h5
./HVmodel/data/origin/bkg_in_SR.h5
./HVmodel/data/origin/bkg_in_SB.h5
./HVmodel/data/origin/val/sig_in_SR.h5
./HVmodel/data/origin/val/sig_in_SB.h5
./HVmodel/data/origin/val/bkg_in_SR.h5
./HVmodel/data/origin/val/bkg_in_SB.h5


# $\eta - \phi$ smearing

In [4]:
def eta_phi_smearing(h5_path, out_h5, std_scale=0.1):

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:

        print(out_h5)

        pt1, pt2 = f_out['J1/pt'][:], f_out['J2/pt'][:]
        eta1, eta2 = f_out['J1/eta'][:], f_out['J2/eta'][:]
        phi1, phi2 = f_out['J1/phi'][:], f_out['J2/phi'][:]
                
        std_1 = np.divide(std_scale, pt1, out=np.zeros_like(pt1), where=(pt1!=0))
        std_2 = np.divide(std_scale, pt2, out=np.zeros_like(pt2), where=(pt2!=0))

        f_out['J1/eta'][:] = np.random.normal(loc=eta1, scale=std_1)
        f_out['J2/eta'][:] = np.random.normal(loc=eta2, scale=std_2)
        f_out['J1/phi'][:] = std_phi(np.random.normal(loc=phi1, scale=std_1))
        f_out['J2/phi'][:] = std_phi(np.random.normal(loc=phi2, scale=std_2))

        # process 
        _, eta1, phi1 = preprocess(f_out['J1/pt'][:], f_out['J1/eta'][:], f_out['J1/phi'][:])
        _, eta2, phi2 = preprocess(f_out['J2/pt'][:], f_out['J2/eta'][:], f_out['J2/phi'][:])

        f_out['J1/eta'][:] = eta1
        f_out['J1/phi'][:] = phi1
        f_out['J2/eta'][:] = eta2
        f_out['J2/phi'][:] = phi2
    

In [5]:
for n in range(1, 4):
    h5_dir = './HVmodel/data/split_val'
    out_dir = f'./HVmodel/data/eta_phi_smearing/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        out_h5 = os.path.join(out_dir, f'{name}.h5')
        eta_phi_smearing(h5_path, out_h5)

./HVmodel/data/eta_phi_smearing/01/sig_in_SR.h5
./HVmodel/data/eta_phi_smearing/01/sig_in_SB.h5
./HVmodel/data/eta_phi_smearing/01/bkg_in_SR.h5
./HVmodel/data/eta_phi_smearing/01/bkg_in_SB.h5
./HVmodel/data/eta_phi_smearing/02/sig_in_SR.h5
./HVmodel/data/eta_phi_smearing/02/sig_in_SB.h5
./HVmodel/data/eta_phi_smearing/02/bkg_in_SR.h5
./HVmodel/data/eta_phi_smearing/02/bkg_in_SB.h5
./HVmodel/data/eta_phi_smearing/03/sig_in_SR.h5
./HVmodel/data/eta_phi_smearing/03/sig_in_SB.h5
./HVmodel/data/eta_phi_smearing/03/bkg_in_SR.h5
./HVmodel/data/eta_phi_smearing/03/bkg_in_SB.h5


# $p_\text{T}$ smearing

In [ ]:
def pt_smearing(h5_path, out_h5):

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)
        # Take pT information
        pt1, pt2 = f_out['J1/pt'][:], f_out['J2/pt'][:]
            
        # Smearing pt
        pt1_tmp = np.random.normal(loc=pt1, scale=pt_scale(pt1))
        pt2_tmp = np.random.normal(loc=pt2, scale=pt_scale(pt2))

        # set negative pt as 0
        pt1_tmp[pt1_tmp < 0] = 0
        pt2_tmp[pt2_tmp < 0] = 0

        f_out['J1/pt'][:] = pt1_tmp
        f_out['J2/pt'][:] = pt2_tmp

        _, eta1, phi1 = preprocess(f_out['J1/pt'][:], f_out['J1/eta'][:], f_out['J1/phi'][:])
        _, eta2, phi2 = preprocess(f_out['J2/pt'][:], f_out['J2/eta'][:], f_out['J2/phi'][:])

        f_out['J1/eta'][:] = eta1
        f_out['J1/phi'][:] = phi1
        f_out['J2/eta'][:] = eta2
        f_out['J2/phi'][:] = phi2

In [3]:
for n in range(1, 6):
    h5_dir = './HVmodel/data/split_val'
    out_dir = f'./HVmodel/data/pt_smearing/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        out_h5 = os.path.join(out_dir, f'{name}.h5')
        pt_smearing(h5_path, out_h5)

./HVmodel/data/pt_smearing/01/sig_in_SR.h5
./HVmodel/data/pt_smearing/01/sig_in_SB.h5
./HVmodel/data/pt_smearing/01/bkg_in_SR.h5
./HVmodel/data/pt_smearing/01/bkg_in_SB.h5
./HVmodel/data/pt_smearing/02/sig_in_SR.h5
./HVmodel/data/pt_smearing/02/sig_in_SB.h5
./HVmodel/data/pt_smearing/02/bkg_in_SR.h5
./HVmodel/data/pt_smearing/02/bkg_in_SB.h5
./HVmodel/data/pt_smearing/03/sig_in_SR.h5
./HVmodel/data/pt_smearing/03/sig_in_SB.h5
./HVmodel/data/pt_smearing/03/bkg_in_SR.h5
./HVmodel/data/pt_smearing/03/bkg_in_SB.h5
./HVmodel/data/pt_smearing/04/sig_in_SR.h5
./HVmodel/data/pt_smearing/04/sig_in_SB.h5
./HVmodel/data/pt_smearing/04/bkg_in_SR.h5
./HVmodel/data/pt_smearing/04/bkg_in_SB.h5
./HVmodel/data/pt_smearing/05/sig_in_SR.h5
./HVmodel/data/pt_smearing/05/sig_in_SB.h5
./HVmodel/data/pt_smearing/05/bkg_in_SR.h5
./HVmodel/data/pt_smearing/05/bkg_in_SB.h5


# Jet rotation

In [3]:
def jet_rotation(h5_path, out_h5, rot_range=np.pi):
    # after pre-process, rotate the jet image with random angle
    # save results in h5 file
    # rot_range: rotation range

    shutil.copyfile(h5_path, out_h5)

    # preprocess the eta and phi
    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)
        _, eta1, phi1 = preprocess(f_out['J1/pt'][:], f_out['J1/eta'][:], f_out['J1/phi'][:])
        _, eta2, phi2 = preprocess(f_out['J2/pt'][:], f_out['J2/eta'][:], f_out['J2/phi'][:])

        f_out['J1/eta'][:] = eta1
        f_out['J1/phi'][:] = phi1
        f_out['J2/eta'][:] = eta2
        f_out['J2/phi'][:] = phi2

        nevent = f_out['EVENT/signal'].shape[0]

        # rotate eta and phi with random angle
        angle = np.random.uniform(-rot_range, rot_range, size=nevent)[:, None]
        eta1_rotat, phi1_rotat = rotation(eta1, phi1, angle)
        angle = np.random.uniform(-rot_range, rot_range, size=nevent)[:, None]
        eta2_rotat, phi2_rotat = rotation(eta2, phi2, angle)

        f_out['J1/eta'][:] = eta1_rotat
        f_out['J1/phi'][:] = phi1_rotat
        f_out['J2/eta'][:] = eta2_rotat
        f_out['J2/phi'][:] = phi2_rotat        

In [4]:
for n in range(6, 34):
    h5_dir = './HVmodel/data/split_val'
    out_dir = f'./HVmodel/data/jet_rotation/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        out_h5 = os.path.join(out_dir, f'{name}.h5')
        jet_rotation(h5_path, out_h5)

./HVmodel/data/jet_rotation/06/sig_in_SR.h5
./HVmodel/data/jet_rotation/06/sig_in_SB.h5
./HVmodel/data/jet_rotation/06/bkg_in_SR.h5
./HVmodel/data/jet_rotation/06/bkg_in_SB.h5
./HVmodel/data/jet_rotation/07/sig_in_SR.h5
./HVmodel/data/jet_rotation/07/sig_in_SB.h5
./HVmodel/data/jet_rotation/07/bkg_in_SR.h5
./HVmodel/data/jet_rotation/07/bkg_in_SB.h5
./HVmodel/data/jet_rotation/08/sig_in_SR.h5
./HVmodel/data/jet_rotation/08/sig_in_SB.h5
./HVmodel/data/jet_rotation/08/bkg_in_SR.h5
./HVmodel/data/jet_rotation/08/bkg_in_SB.h5
./HVmodel/data/jet_rotation/09/sig_in_SR.h5
./HVmodel/data/jet_rotation/09/sig_in_SB.h5
./HVmodel/data/jet_rotation/09/bkg_in_SR.h5
./HVmodel/data/jet_rotation/09/bkg_in_SB.h5
./HVmodel/data/jet_rotation/10/sig_in_SR.h5
./HVmodel/data/jet_rotation/10/sig_in_SB.h5
./HVmodel/data/jet_rotation/10/bkg_in_SR.h5
./HVmodel/data/jet_rotation/10/bkg_in_SB.h5
./HVmodel/data/jet_rotation/11/sig_in_SR.h5
./HVmodel/data/jet_rotation/11/sig_in_SB.h5
./HVmodel/data/jet_rotation/11/b

In [4]:
for n in range(34, 51):
    h5_dir = './HVmodel/data/split_val'
    out_dir = f'./HVmodel/data/jet_rotation/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        out_h5 = os.path.join(out_dir, f'{name}.h5')
        jet_rotation(h5_path, out_h5)

./HVmodel/data/jet_rotation/34/sig_in_SR.h5
./HVmodel/data/jet_rotation/34/sig_in_SB.h5
./HVmodel/data/jet_rotation/34/bkg_in_SR.h5
./HVmodel/data/jet_rotation/34/bkg_in_SB.h5
./HVmodel/data/jet_rotation/35/sig_in_SR.h5
./HVmodel/data/jet_rotation/35/sig_in_SB.h5
./HVmodel/data/jet_rotation/35/bkg_in_SR.h5
./HVmodel/data/jet_rotation/35/bkg_in_SB.h5
./HVmodel/data/jet_rotation/36/sig_in_SR.h5
./HVmodel/data/jet_rotation/36/sig_in_SB.h5
./HVmodel/data/jet_rotation/36/bkg_in_SR.h5
./HVmodel/data/jet_rotation/36/bkg_in_SB.h5
./HVmodel/data/jet_rotation/37/sig_in_SR.h5
./HVmodel/data/jet_rotation/37/sig_in_SB.h5
./HVmodel/data/jet_rotation/37/bkg_in_SR.h5
./HVmodel/data/jet_rotation/37/bkg_in_SB.h5
./HVmodel/data/jet_rotation/38/sig_in_SR.h5
./HVmodel/data/jet_rotation/38/sig_in_SB.h5
./HVmodel/data/jet_rotation/38/bkg_in_SR.h5
./HVmodel/data/jet_rotation/38/bkg_in_SB.h5
./HVmodel/data/jet_rotation/39/sig_in_SR.h5
./HVmodel/data/jet_rotation/39/sig_in_SB.h5
./HVmodel/data/jet_rotation/39/b

# $p_\text{T}$ smearing + Jet rotation

In [5]:
def pt_smearing_jet_rotation(h5_path, out_h5, rot_range=np.pi):
    # after pre-process, rotate the jet image with random angle
    # rot_range: rotation range

    shutil.copyfile(h5_path, out_h5)
    
    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)     
        
        # Take pT, information
        pt1, pt2 = f_out['J1/pt'][:], f_out['J2/pt'][:]

        # Smearing pt
        pt1_tmp = np.random.normal(loc=pt1, scale=pt_scale(pt1))
        pt2_tmp = np.random.normal(loc=pt2, scale=pt_scale(pt2))

        # set negative pt as 0
        pt1_tmp[pt1_tmp < 0] = 0
        pt2_tmp[pt2_tmp < 0] = 0

        f_out['J1/pt'][:] = pt1_tmp
        f_out['J2/pt'][:] = pt2_tmp

        _, eta1_tmp, phi1_tmp = preprocess(f_out['J1/pt'][:], f_out['J1/eta'][:], f_out['J1/phi'][:])
        _, eta2_tmp, phi2_tmp = preprocess(f_out['J2/pt'][:], f_out['J2/eta'][:], f_out['J2/phi'][:])

        nevent = f_out['EVENT/signal'].shape[0]

        # rotate eta and phi with random angle
        angle = np.random.uniform(-rot_range, rot_range, size=nevent)[:, None]
        eta1_rotat, phi1_rotat = rotation(eta1_tmp, phi1_tmp, angle)
        angle = np.random.uniform(-rot_range, rot_range, size=nevent)[:, None]
        eta2_rotat, phi2_rotat = rotation(eta2_tmp, phi2_tmp, angle)

        f_out['J1/eta'][:] = eta1_rotat
        f_out['J1/phi'][:] = phi1_rotat
        f_out['J2/eta'][:] = eta2_rotat
        f_out['J2/phi'][:] = phi2_rotat


In [6]:
for n in range(34, 51):
    h5_dir = './HVmodel/data/split_val'
    out_dir = f'./HVmodel/data/pt_smearing_jet_rotation/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        out_h5 = os.path.join(out_dir, f'{name}.h5')
        pt_smearing_jet_rotation(h5_path, out_h5)

./HVmodel/data/pt_smearing_jet_rotation/34/sig_in_SR.h5


./HVmodel/data/pt_smearing_jet_rotation/34/sig_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/34/bkg_in_SR.h5
./HVmodel/data/pt_smearing_jet_rotation/34/bkg_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/35/sig_in_SR.h5
./HVmodel/data/pt_smearing_jet_rotation/35/sig_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/35/bkg_in_SR.h5
./HVmodel/data/pt_smearing_jet_rotation/35/bkg_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/36/sig_in_SR.h5
./HVmodel/data/pt_smearing_jet_rotation/36/sig_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/36/bkg_in_SR.h5
./HVmodel/data/pt_smearing_jet_rotation/36/bkg_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/37/sig_in_SR.h5
./HVmodel/data/pt_smearing_jet_rotation/37/sig_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/37/bkg_in_SR.h5
./HVmodel/data/pt_smearing_jet_rotation/37/bkg_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/38/sig_in_SR.h5
./HVmodel/data/pt_smearing_jet_rotation/38/sig_in_SB.h5
./HVmodel/data/pt_smearing_jet_rotation/38/bkg_i

# $\eta - \phi$ smearing + $p_\text{T}$ smearing + Jet rotation

In [7]:
def eta_phi_pt_smearing_jet_rotation(h5_path, out_h5, std_scale=0.1, rot_range=np.pi):
    # after pre-process, rotate the jet image with random angle
    # rot_range: rotation range

    shutil.copyfile(h5_path, out_h5)
    
    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)     
        
        # Take pT, eta, phi information
        pt1, pt2 = f_out['J1/pt'][:], f_out['J2/pt'][:]
        eta1, eta2 = f_out['J1/eta'][:], f_out['J2/eta'][:]
        phi1, phi2 = f_out['J1/phi'][:], f_out['J2/phi'][:]

        # ete and phi smearing
        std_1 = np.divide(std_scale, pt1, out=np.zeros_like(pt1), where=(pt1!=0))
        std_2 = np.divide(std_scale, pt2, out=np.zeros_like(pt2), where=(pt2!=0))

        f_out['J1/eta'][:] = np.random.normal(loc=eta1, scale=std_1)
        f_out['J2/eta'][:] = np.random.normal(loc=eta2, scale=std_2)
        f_out['J1/phi'][:] = std_phi(np.random.normal(loc=phi1, scale=std_1))
        f_out['J2/phi'][:] = std_phi(np.random.normal(loc=phi2, scale=std_2))

        # Smearing pt
        pt1_tmp = np.random.normal(loc=pt1, scale=pt_scale(pt1))
        pt2_tmp = np.random.normal(loc=pt2, scale=pt_scale(pt2))

        # set negative pt as 0
        pt1_tmp[pt1_tmp < 0] = 0
        pt2_tmp[pt2_tmp < 0] = 0

        f_out['J1/pt'][:] = pt1_tmp
        f_out['J2/pt'][:] = pt2_tmp

        _, eta1_tmp, phi1_tmp = preprocess(f_out['J1/pt'][:], f_out['J1/eta'][:], f_out['J1/phi'][:])
        _, eta2_tmp, phi2_tmp = preprocess(f_out['J2/pt'][:], f_out['J2/eta'][:], f_out['J2/phi'][:])

        nevent = f_out['EVENT/signal'].shape[0]

        # rotate eta and phi with random angle
        angle = np.random.uniform(-rot_range, rot_range, size=nevent)[:, None]
        eta1_rotat, phi1_rotat = rotation(eta1_tmp, phi1_tmp, angle)
        angle = np.random.uniform(-rot_range, rot_range, size=nevent)[:, None]
        eta2_rotat, phi2_rotat = rotation(eta2_tmp, phi2_tmp, angle)

        f_out['J1/eta'][:] = eta1_rotat
        f_out['J1/phi'][:] = phi1_rotat
        f_out['J2/eta'][:] = eta2_rotat
        f_out['J2/phi'][:] = phi2_rotat


In [8]:
for n in range(23, 34):
    h5_dir = './HVmodel/data/split_val'
    out_dir = f'./HVmodel/data/eta_phi_pt_smearing_jet_rotation/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        out_h5 = os.path.join(out_dir, f'{name}.h5')
        eta_phi_pt_smearing_jet_rotation(h5_path, out_h5)

./HVmodel/data/eta_phi_pt_smearing_jet_rotation/23/sig_in_SR.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/23/sig_in_SB.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/23/bkg_in_SR.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/23/bkg_in_SB.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/24/sig_in_SR.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/24/sig_in_SB.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/24/bkg_in_SR.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/24/bkg_in_SB.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/25/sig_in_SR.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/25/sig_in_SB.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/25/bkg_in_SR.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/25/bkg_in_SB.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/26/sig_in_SR.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/26/sig_in_SB.h5
./HVmodel/data/eta_phi_pt_smearing_jet_rotation/26/bkg_in_SR.h5
./HVmodel/data/eta_phi_pt_smearing_jet_r

# Only shifting to $p_\text{T}$ center

In [3]:
def to_pt_center_h5(h5_path, out_h5):

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)
        _, eta1, phi1 = preprocess_only_centralization(f_out['J1/pt'][:], f_out['J1/eta'][:], f_out['J1/phi'][:])
        _, eta2, phi2 = preprocess_only_centralization(f_out['J2/pt'][:], f_out['J2/eta'][:], f_out['J2/phi'][:])

        f_out['J1/eta'][:] = eta1
        f_out['J1/phi'][:] = phi1
        f_out['J2/eta'][:] = eta2
        f_out['J2/phi'][:] = phi2

In [6]:
for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = f'./HVmodel/data/split_val/{name}.h5'
    out_h5 = f'./HVmodel/data/pt_center/{name}.h5'
    to_pt_center_h5(h5_path, out_h5)

for new_name, old_name in zip(['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB'], 
                              ['sig-val_in_SR', 'sig-val_in_SB', 'bkg-val_in_SR', 'bkg-val_in_SB']):
    h5_path = f'./HVmodel/data/split_val/{old_name}.h5'
    out_h5 = f'./HVmodel/data/pt_center/val/{new_name}.h5'
    to_pt_center_h5(h5_path, out_h5)

./HVmodel/data/pt_center/sig_in_SR.h5
./HVmodel/data/pt_center/sig_in_SB.h5
./HVmodel/data/pt_center/bkg_in_SR.h5
./HVmodel/data/pt_center/bkg_in_SB.h5
./HVmodel/data/pt_center/val/sig_in_SR.h5
./HVmodel/data/pt_center/val/sig_in_SB.h5
./HVmodel/data/pt_center/val/bkg_in_SR.h5
./HVmodel/data/pt_center/val/bkg_in_SB.h5


# $m_{jj}$ normalization

In [3]:
def mjj_normalization(h5_path, out_h5):

    shutil.copyfile(h5_path, out_h5)

    center_of_SR = 5100

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)
        # Take pT information
        pt1, pt2 = f_out['J1/pt'][:], f_out['J2/pt'][:]

        # Take mjj information
        mjj = f_out['EVENT/Mjj'][:][:, None]

        # Normalize pT
        f_out['J1/pt'][:] = pt1 / mjj * center_of_SR
        f_out['J2/pt'][:] = pt2 / mjj * center_of_SR

        _, eta1, phi1 = preprocess(f_out['J1/pt'][:], f_out['J1/eta'][:], f_out['J1/phi'][:])
        _, eta2, phi2 = preprocess(f_out['J2/pt'][:], f_out['J2/eta'][:], f_out['J2/phi'][:])

        f_out['J1/eta'][:] = eta1
        f_out['J1/phi'][:] = phi1
        f_out['J2/eta'][:] = eta2
        f_out['J2/phi'][:] = phi2

In [4]:
for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = f'./HVmodel/data/split_val/{name}.h5'
    out_h5 = f'./HVmodel/data/mjj_normalization/{name}.h5'
    mjj_normalization(h5_path, out_h5)

for new_name, old_name in zip(['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB'], 
                              ['sig-val_in_SR', 'sig-val_in_SB', 'bkg-val_in_SR', 'bkg-val_in_SB']):
    h5_path = f'./HVmodel/data/split_val/{old_name}.h5'
    out_h5 = f'./HVmodel/data/mjj_normalization/val/{new_name}.h5'
    mjj_normalization(h5_path, out_h5)

./HVmodel/data/pt_normalization/sig_in_SR.h5
./HVmodel/data/pt_normalization/sig_in_SB.h5
./HVmodel/data/pt_normalization/bkg_in_SR.h5
./HVmodel/data/pt_normalization/bkg_in_SB.h5
./HVmodel/data/pt_normalization/val/sig_in_SR.h5
./HVmodel/data/pt_normalization/val/sig_in_SB.h5
./HVmodel/data/pt_normalization/val/bkg_in_SR.h5
./HVmodel/data/pt_normalization/val/bkg_in_SB.h5


In [6]:
name = 'mix_sample_test-4'
h5_path = f'./HVmodel/data/split_val/{name}.h5'
out_h5 = f'./HVmodel/data/mjj_normalization/testing/{name}.h5'
mjj_normalization(h5_path, out_h5)

./HVmodel/data/pt_normalization/testing/mix_sample_test-4.h5


# Take invariant mass

In [1]:
def get_mjj_from_h5(h5_path, out_npy):

    with h5py.File(h5_path, 'r') as f:
        mjj = f['EVENT/Mjj'][:]
        np.save(out_npy, mjj)

In [4]:
for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = f'./HVmodel/data/origin/{name}.h5'
    out_npy = f'./HVmodel/data/origin/{name}-mjj.npy'
    get_mjj_from_h5(h5_path, out_npy)

# Testing sample

In [3]:
name = 'mix_sample_test-mjj'
h5_path = f'./HVmodel/data/split_val/{name}.h5'
out_h5 = f'./HVmodel/data/split_val/testing/{name}.h5'
to_origin_h5(h5_path, out_h5)

./HVmodel/data/split_val/testing/mix_sample_test-mjj.h5


In [9]:
name = 'mix_sample_test-2'
h5_path = f'./HVmodel/data/split_val/{name}.h5'
out_h5 = f'./HVmodel/data/split_val/testing/{name}.h5'
to_origin_h5(h5_path, out_h5)

./HVmodel/data/split_val/testing/mix_sample_test-2.h5


In [4]:
name = 'mix_sample_test-3'
h5_path = f'./HVmodel/data/split_val/{name}.h5'
out_h5 = f'./HVmodel/data/split_val/testing/{name}.h5'
to_origin_h5(h5_path, out_h5)

./HVmodel/data/split_val/testing/mix_sample_test-3.h5


In [5]:
name = 'mix_sample_test-4'
h5_path = f'./HVmodel/data/split_val/{name}.h5'
out_h5 = f'./HVmodel/data/split_val/testing/{name}.h5'
to_origin_h5(h5_path, out_h5)

./HVmodel/data/split_val/testing/mix_sample_test-4.h5


# Pixelization

In [6]:
res = 75
h5_dir = './HVmodel/data/origin'
npy_dir = f'./HVmodel/data/origin/{res}x{res}'

# create output directory
if not os.path.exists(npy_dir):
    os.makedirs(npy_dir)

for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = os.path.join(h5_dir, f'{name}.h5')
    npy_path = os.path.join(npy_dir, f'{name}.npy')
    cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = './HVmodel/data/origin/val'
npy_dir = f'./HVmodel/data/origin/{res}x{res}/val'

# create output directory
if not os.path.exists(npy_dir):
    os.makedirs(npy_dir)

for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = os.path.join(h5_dir, f'{name}.h5')
    npy_path = os.path.join(npy_dir, f'{name}.npy')
    cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelization.py ./HVmodel/data/origin/sig_in_SR.h5 ./HVmodel/data/origin/75x75/sig_in_SR.npy 75 &
python pixelization.py ./HVmodel/data/origin/sig_in_SB.h5 ./HVmodel/data/origin/75x75/sig_in_SB.npy 75 &
python pixelization.py ./HVmodel/data/origin/bkg_in_SR.h5 ./HVmodel/data/origin/75x75/bkg_in_SR.npy 75 &
python pixelization.py ./HVmodel/data/origin/bkg_in_SB.h5 ./HVmodel/data/origin/75x75/bkg_in_SB.npy 75 &
python pixelization.py ./HVmodel/data/origin/val/sig_in_SR.h5 ./HVmodel/data/origin/75x75/val/sig_in_SR.npy 75 &
python pixelization.py ./HVmodel/data/origin/val/sig_in_SB.h5 ./HVmodel/data/origin/75x75/val/sig_in_SB.npy 75 &
python pixelization.py ./HVmodel/data/origin/val/bkg_in_SR.h5 ./HVmodel/data/origin/75x75/val/bkg_in_SR.npy 75 &
python pixelization.py ./HVmodel/data/origin/val/bkg_in_SB.h5 ./HVmodel/data/origin/75x75/val/bkg_in_SB.npy 75 &


## Only shifting to $p_\text{T}$ center

In [7]:
res = 25
h5_dir = './HVmodel/data/pt_center'
out_dir = f'./HVmodel/data/pt_center/{res}x{res}'

for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = os.path.join(h5_dir, f'{name}.h5')
    npy_path = os.path.join(out_dir, f'{name}.npy')

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = './HVmodel/data/pt_center/val'
out_dir = f'./HVmodel/data/pt_center/{res}x{res}/val'

for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = os.path.join(h5_dir, f'{name}.h5')
    npy_path = os.path.join(out_dir, f'{name}.npy')
    
    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelization.py ./HVmodel/data/pt_center/sig_in_SR.h5 ./HVmodel/data/pt_center/25x25/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_center/sig_in_SB.h5 ./HVmodel/data/pt_center/25x25/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_center/bkg_in_SR.h5 ./HVmodel/data/pt_center/25x25/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_center/bkg_in_SB.h5 ./HVmodel/data/pt_center/25x25/bkg_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_center/val/sig_in_SR.h5 ./HVmodel/data/pt_center/25x25/val/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_center/val/sig_in_SB.h5 ./HVmodel/data/pt_center/25x25/val/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_center/val/bkg_in_SR.h5 ./HVmodel/data/pt_center/25x25/val/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_center/val/bkg_in_SB.h5 ./HVmodel/data/pt_center/25x25/val/bkg_in_SB.npy 25 &


In [6]:
res = 25
for n in range(1, 4):

    h5_dir = f'./HVmodel/data/eta_phi_smearing/{n:02}'
    out_dir = f'./HVmodel/data/eta_phi_smearing/{res}x{res}/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        npy_path = os.path.join(out_dir, f'{name}.npy')
        cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
        print(cmd)


python pixelization.py ./HVmodel/data/eta_phi_smearing/01/sig_in_SR.h5 ./HVmodel/data/eta_phi_smearing/25x25/01/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/eta_phi_smearing/01/sig_in_SB.h5 ./HVmodel/data/eta_phi_smearing/25x25/01/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/eta_phi_smearing/01/bkg_in_SR.h5 ./HVmodel/data/eta_phi_smearing/25x25/01/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/eta_phi_smearing/01/bkg_in_SB.h5 ./HVmodel/data/eta_phi_smearing/25x25/01/bkg_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/eta_phi_smearing/02/sig_in_SR.h5 ./HVmodel/data/eta_phi_smearing/25x25/02/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/eta_phi_smearing/02/sig_in_SB.h5 ./HVmodel/data/eta_phi_smearing/25x25/02/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/eta_phi_smearing/02/bkg_in_SR.h5 ./HVmodel/data/eta_phi_smearing/25x25/02/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/eta_phi_smearing/02/bkg_in_SB.h5 ./HVmodel/da

In [ ]:
res = 25
for n in range(1, 6):

    h5_dir = f'./HVmodel/data/pt_smearing/{n:02}'
    out_dir = f'./HVmodel/data/pt_smearing/{res}x{res}/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        npy_path = os.path.join(out_dir, f'{name}.npy')
        cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
        print(cmd)


python pixelization.py ./HVmodel/data/pt_smearing/01/sig_in_SR.h5 ./HVmodel/data/pt_smearing/25x25/01/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing/01/sig_in_SB.h5 ./HVmodel/data/pt_smearing/25x25/01/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing/01/bkg_in_SR.h5 ./HVmodel/data/pt_smearing/25x25/01/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing/01/bkg_in_SB.h5 ./HVmodel/data/pt_smearing/25x25/01/bkg_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing/02/sig_in_SR.h5 ./HVmodel/data/pt_smearing/25x25/02/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing/02/sig_in_SB.h5 ./HVmodel/data/pt_smearing/25x25/02/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing/02/bkg_in_SR.h5 ./HVmodel/data/pt_smearing/25x25/02/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing/02/bkg_in_SB.h5 ./HVmodel/data/pt_smearing/25x25/02/bkg_in_SB.npy 25 &
python pixelization.py ./HVmodel

## Jet rotation

In [7]:
res = 25
for n in range(34, 51):

    h5_dir = f'./HVmodel/data/jet_rotation/{n:02}'
    out_dir = f'./HVmodel/data/jet_rotation/{res}x{res}/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        npy_path = os.path.join(out_dir, f'{name}.npy')
        cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
        print(cmd)


python pixelization.py ./HVmodel/data/jet_rotation/34/sig_in_SR.h5 ./HVmodel/data/jet_rotation/25x25/34/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/jet_rotation/34/sig_in_SB.h5 ./HVmodel/data/jet_rotation/25x25/34/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/jet_rotation/34/bkg_in_SR.h5 ./HVmodel/data/jet_rotation/25x25/34/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/jet_rotation/34/bkg_in_SB.h5 ./HVmodel/data/jet_rotation/25x25/34/bkg_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/jet_rotation/35/sig_in_SR.h5 ./HVmodel/data/jet_rotation/25x25/35/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/jet_rotation/35/sig_in_SB.h5 ./HVmodel/data/jet_rotation/25x25/35/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/jet_rotation/35/bkg_in_SR.h5 ./HVmodel/data/jet_rotation/25x25/35/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/jet_rotation/35/bkg_in_SB.h5 ./HVmodel/data/jet_rotation/25x25/35/bkg_in_SB.npy 25 &
python pixelizat

## $p_\text{T}$ smearing + Jet rotation

In [3]:
res = 25
for n in range(34, 51):

    h5_dir = f'./HVmodel/data/pt_smearing_jet_rotation/{n:02}'
    out_dir = f'./HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}'

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
        h5_path = os.path.join(h5_dir, f'{name}.h5')
        npy_path = os.path.join(out_dir, f'{name}.npy')
        cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
        print(cmd)


python pixelization.py ./HVmodel/data/pt_smearing_jet_rotation/34/sig_in_SR.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/34/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing_jet_rotation/34/sig_in_SB.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/34/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing_jet_rotation/34/bkg_in_SR.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/34/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing_jet_rotation/34/bkg_in_SB.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/34/bkg_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing_jet_rotation/35/sig_in_SR.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/35/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing_jet_rotation/35/sig_in_SB.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/35/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_smearing_jet_rotation/35/bkg_in_SR.h5 ./HVmodel/data/pt_smearing_jet_rotation

## $m_{jj}$ normalization

In [7]:
res = 25
h5_dir = './HVmodel/data/mjj_normalization'
out_dir = f'./HVmodel/data/mjj_normalization/{res}x{res}'

for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = os.path.join(h5_dir, f'{name}.h5')
    npy_path = os.path.join(out_dir, f'{name}.npy')

    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = './HVmodel/data/mjj_normalization/val'
out_dir = f'./HVmodel/data/mjj_normalization/{res}x{res}/val'

for name in ['sig_in_SR', 'sig_in_SB', 'bkg_in_SR', 'bkg_in_SB']:
    h5_path = os.path.join(h5_dir, f'{name}.h5')
    npy_path = os.path.join(out_dir, f'{name}.npy')
    
    # create output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
    print(cmd)

h5_dir = './HVmodel/data/mjj_normalization/testing'
npy_dir = f'./HVmodel/data/mjj_normalization/{res}x{res}'

name = 'mix_sample_test-4'
h5_path = os.path.join(h5_dir, f'{name}.h5')
npy_path = os.path.join(npy_dir, f'{name}.npy')
cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
print(cmd)

python pixelization.py ./HVmodel/data/pt_normalization/sig_in_SR.h5 ./HVmodel/data/pt_normalization/25x25/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_normalization/sig_in_SB.h5 ./HVmodel/data/pt_normalization/25x25/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_normalization/bkg_in_SR.h5 ./HVmodel/data/pt_normalization/25x25/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_normalization/bkg_in_SB.h5 ./HVmodel/data/pt_normalization/25x25/bkg_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_normalization/val/sig_in_SR.h5 ./HVmodel/data/pt_normalization/25x25/val/sig_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_normalization/val/sig_in_SB.h5 ./HVmodel/data/pt_normalization/25x25/val/sig_in_SB.npy 25 &
python pixelization.py ./HVmodel/data/pt_normalization/val/bkg_in_SR.h5 ./HVmodel/data/pt_normalization/25x25/val/bkg_in_SR.npy 25 &
python pixelization.py ./HVmodel/data/pt_normalization/val/bkg_in_SB.h5 ./HVmodel/data/pt_normalizati

## Testing sample

In [4]:
res = 25
h5_dir = './HVmodel/data/split_val/testing'
npy_dir = f'./HVmodel/data/split_val/{res}x{res}'

name = 'mix_sample_test-mjj'
h5_path = os.path.join(h5_dir, f'{name}.h5')
npy_path = os.path.join(npy_dir, f'{name}.npy')
cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
print(cmd)

python pixelization.py ./HVmodel/data/split_val/testing/mix_sample_test-mjj.h5 ./HVmodel/data/split_val/25x25/mix_sample_test-mjj.npy 25 &


In [10]:
res = 25
h5_dir = './HVmodel/data/split_val/testing'
npy_dir = f'./HVmodel/data/split_val/{res}x{res}'

name = 'mix_sample_test-2'
h5_path = os.path.join(h5_dir, f'{name}.h5')
npy_path = os.path.join(npy_dir, f'{name}.npy')
cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
print(cmd)

python pixelization.py ./HVmodel/data/split_val/testing/mix_sample_test-2.h5 ./HVmodel/data/split_val/25x25/mix_sample_test-2.npy 25 &


In [5]:
res = 25
h5_dir = './HVmodel/data/split_val/testing'
npy_dir = f'./HVmodel/data/split_val/{res}x{res}'

name = 'mix_sample_test-3'
h5_path = os.path.join(h5_dir, f'{name}.h5')
npy_path = os.path.join(npy_dir, f'{name}.npy')
cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
print(cmd)

python pixelization.py ./HVmodel/data/split_val/testing/mix_sample_test-3.h5 ./HVmodel/data/split_val/25x25/mix_sample_test-3.npy 25 &


In [6]:
res = 25
h5_dir = './HVmodel/data/split_val/testing'
npy_dir = f'./HVmodel/data/split_val/{res}x{res}'

name = 'mix_sample_test-4'
h5_path = os.path.join(h5_dir, f'{name}.h5')
npy_path = os.path.join(npy_dir, f'{name}.npy')
cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
print(cmd)

python pixelization.py ./HVmodel/data/split_val/testing/mix_sample_test-4.h5 ./HVmodel/data/split_val/25x25/mix_sample_test-4.npy 25 &


In [3]:
res = 75
h5_dir = './HVmodel/data/split_val/testing'
npy_dir = f'./HVmodel/data/split_val/{res}x{res}'

name = 'mix_sample_test'
h5_path = os.path.join(h5_dir, f'{name}.h5')
npy_path = os.path.join(npy_dir, f'{name}.npy')
cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
print(cmd)


python pixelization.py ./HVmodel/data/split_val/testing/mix_sample_test.h5 ./HVmodel/data/split_val/75x75/mix_sample_test.npy 75 &


In [3]:
res = 75
h5_dir = './HVmodel/data/split_val/testing'
npy_dir = f'./HVmodel/data/split_val/{res}x{res}'

name = 'mix_sample_test-SB'
h5_path = os.path.join(h5_dir, f'{name}.h5')
npy_path = os.path.join(npy_dir, f'{name}.npy')
cmd = f'python pixelization.py {h5_path} {npy_path} {res} &'
print(cmd)


python pixelization.py ./HVmodel/data/split_val/testing/mix_sample_test-SB.h5 ./HVmodel/data/split_val/75x75/mix_sample_test-SB.npy 75 &
